## Import bibliotek

In [1]:
import pandas as pd
import datetime
from sklearn import preprocessing

### Import pliku

In [2]:
df = pd.read_csv('DanePogoda.csv',encoding='ISO-8859-2',sep=';')
# plik ma problem z kodowaniem. Zadeklarowanie ISO-8859-2 pozwala na wczytanie, ale polskie znaki są pomijane.

In [3]:
df.head()

,Stacja,Data,Godzina,zjawisko,Zachmurzenia,Cisnienie
0,12570,01.06.2020,00:00:00,Mga,6,1017.7
1,12560,01.06.2020,00:00:00,Mga,6,1018.6
2,12570,01.06.2020,01:00:00,Mga,2,1017.6
3,12560,01.06.2020,01:00:00,Mga,5,1018.6
4,12566,01.06.2020,01:00:00,Mga,4,1018.2


### Przygotowanie

#### Transformacja pól

In [4]:
# konwersja godziny
df['Godzina'] = pd.to_datetime(df['Godzina'], format='%H:%M:%S').dt.hour

#### Przygotowanie mapowania

In [5]:
def prepareMappingDict(inputLabel):
    '''
    Funkcja generuje słownik mapowania na podstawie zadanych etykiet i przypisanych przedziałów 
    
    input_label: Dict[str, List[Union[List]]] - definicja mapowania w postaci:
        'etykieta': [[zakres_od, zakres_do], ..., [zakres_od, zakres_do]]
    '''
    recoded = {}
    for k,v in inputLabel.items():               # dla każdego itema (klucz, wartość) w słowniku
        for vv in v:                             # dla każdej listy przedziału w liście wartości 
            for l in range(vv[0],vv[1]+1):       # wygeneruj listę wartości od 'zakres_od' do 'zakres_do' 
                recoded[l] = k                   # i dla każdej wartości przypisz ten sam klucz
    return recoded

hourDict = {
    'wieczór': [[18,20]],
    'rano': [[6,17]],
    'noc' : [[21,23],[0,5]]
}

skyDict = {
    'brak': [[0,0]],
    'małe': [[1,3]],
    'umiarkowane': [[4,5]],
    'duże': [[6,7]],
    'pełne': [[8,8]]
}

print('Mapowanie dla Zachmurzenia\n')
print(prepareMappingDict(skyDict))

print('\n\nMapowanie dla Godziny\n')
print(dict(sorted(prepareMappingDict(hourDict).items())))

#pd.DataFrame.from_dict(prepareMappingDict(hourDict), orient='index', columns=['mapping'])

Mapowanie dla Zachmurzenia

{0: 'brak', 1: 'małe', 2: 'małe', 3: 'małe', 4: 'umiarkowane', 5: 'umiarkowane', 6: 'duże', 7: 'duże', 8: 'pełne'}


Mapowanie dla Godziny

{0: 'noc', 1: 'noc', 2: 'noc', 3: 'noc', 4: 'noc', 5: 'noc', 6: 'rano', 7: 'rano', 8: 'rano', 9: 'rano', 10: 'rano', 11: 'rano', 12: 'rano', 13: 'rano', 14: 'rano', 15: 'rano', 16: 'rano', 17: 'rano', 18: 'wieczór', 19: 'wieczór', 20: 'wieczór', 21: 'noc', 22: 'noc', 23: 'noc'}


#### Funkcja do mapowania

In [6]:
def performMapping(x, mapping_dict):
    '''
    Przeprowadź mapowanie danej kolumny na podstawie słownika mapowań. Zasygnalizuj bład, jeśli pojawia się wartość bez mapowania.
    x - Seria Pandas, kolumna którą mapujemy
    mapping_dict - słownik mapowań
    '''
    try:
        return prepareMappingDict(mapping_dict)[x]
    except KeyError as e:
        raise KeyError(f'Wartość {x} nie jest zmapowana w słowniku {mapping_dict=}')

#### Mapowanie

In [7]:
# Mapowanie - Zachmurzenie
df['Stopień zachmurzenia'] = df['Zachmurzenia'].apply(performMapping, mapping_dict=skyDict)

In [8]:
# Mapowanie - Godzina
df['Pora dnia'] = df['Godzina'].apply(performMapping, mapping_dict=hourDict)

In [9]:
# ## Sprawdzenie
#df[['Godzina','Pora dnia']].drop_duplicates().sort_values(by='Godzina')

df[['Zachmurzenia','Stopień zachmurzenia']].drop_duplicates().sort_values(by='Zachmurzenia')

,Zachmurzenia,Stopień zachmurzenia
7,0,brak
17,1,małe
2,2,małe
15,3,małe
4,4,umiarkowane
3,5,umiarkowane
0,6,duże
8,7,duże
29,8,pełne


#### Normalizacja

In [10]:
# Normalizacja Ciśnienie
df['Cisnienie norm'] = preprocessing.MinMaxScaler(feature_range=(0,1)).fit_transform(df['Cisnienie'].values.reshape(-1,1))

# sprawdzenie
df['Cisnienie norm'].describe()

count    11274.000000
mean         0.583396
std          0.150591
min          0.000000
25%          0.501326
50%          0.572944
75%          0.681698
max          1.000000
Name: Cisnienie norm, dtype: float64

#### One-hot encoding

In [11]:
df = pd.get_dummies(df, columns=['zjawisko'], prefix='zjawisko')
df = pd.get_dummies(df, columns=['Pora dnia'], prefix='godzina')
df = pd.get_dummies(df, columns=['Stopień zachmurzenia'], prefix='zachmurzenie')

In [12]:
zjawisko_columns = df.columns.str.match('zjawisko')
df.iloc[:,zjawisko_columns] = df.iloc[:,zjawisko_columns].apply(lambda x: x.map({1:'Tak',0:'Nie'}))

### Podgląd

In [13]:
df.head(50)

,Stacja,Data,Godzina,Zachmurzenia,Cisnienie,Cisnienie norm,zjawisko_Burza,zjawisko_Deszcz,zjawisko_Mga,godzina_noc,godzina_rano,godzina_wieczór,zachmurzenie_brak,zachmurzenie_duże,zachmurzenie_małe,zachmurzenie_pełne,zachmurzenie_umiarkowane
0,12570,01.06.2020,0,6,1017.7,0.721485,Nie,Nie,Tak,1,0,0,0,1,0,0,0
1,12560,01.06.2020,0,6,1018.6,0.745358,Nie,Nie,Tak,1,0,0,0,1,0,0,0
2,12570,01.06.2020,1,2,1017.6,0.718833,Nie,Nie,Tak,1,0,0,0,0,1,0,0
3,12560,01.06.2020,1,5,1018.6,0.745358,Nie,Nie,Tak,1,0,0,0,0,0,0,1
4,12566,01.06.2020,1,4,1018.2,0.734748,Nie,Nie,Tak,1,0,0,0,0,0,0,1
5,12520,01.06.2020,2,2,1021.3,0.816976,Nie,Nie,Tak,1,0,0,0,0,1,0,0
6,12560,01.06.2020,2,6,1018.5,0.742706,Nie,Nie,Tak,1,0,0,0,1,0,0,0
7,12595,01.06.2020,2,0,1015.9,0.673740,Nie,Tak,Nie,1,0,0,1,0,0,0,0
8,12566,01.06.2020,2,7,1018.0,0.729443,Nie,Nie,Tak,1,0,0,0,1,0,0,0
9,12500,01.06.2020,3,0,1023.2,0.867374,Nie,Nie,Tak,1,0,0,1,0,0,0,0


#### Usunięcie zbędnych kolumn

In [14]:
df_clean = df.drop(['Godzina','Zachmurzenia','Cisnienie'], axis=1)
df_clean.head()

,Stacja,Data,Cisnienie norm,zjawisko_Burza,zjawisko_Deszcz,zjawisko_Mga,godzina_noc,godzina_rano,godzina_wieczór,zachmurzenie_brak,zachmurzenie_duże,zachmurzenie_małe,zachmurzenie_pełne,zachmurzenie_umiarkowane
0,12570,01.06.2020,0.721485,Nie,Nie,Tak,1,0,0,0,1,0,0,0
1,12560,01.06.2020,0.745358,Nie,Nie,Tak,1,0,0,0,1,0,0,0
2,12570,01.06.2020,0.718833,Nie,Nie,Tak,1,0,0,0,0,1,0,0
3,12560,01.06.2020,0.745358,Nie,Nie,Tak,1,0,0,0,0,0,0,1
4,12566,01.06.2020,0.734748,Nie,Nie,Tak,1,0,0,0,0,0,0,1
